In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 3197
valuation_date = date.today()
#valuation_date = date(2024, 9, 1)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

すかいらーく HD(3197)の銘柄情報

終値: 2431.5円(2024年12月18日)
予想配当利回り: 0.72%(2024年12月18日)
予想PER: 42.55倍(2024年12月18日)
実績PBR: 3.26倍(2024年12月18日)
自己資本比率: 38.1%(2024年12月18日)
予想ROE: 7.67%(2024年12月18日)
予想ROA: 2.92%(2024年12月18日)
時価総額: 5531.7億円(2024年12月18日)

3197(すかいらーく HD)の四季報データ履歴

発行日: 2024年09月13日
【大幅増額】
  店舗純増約30(前期純減80)。ファミレスは看板の視認性向上効果発現などで客足想定超。食材費増加こなす。のれん除却損減り、営業益上振れ。配当増額。25年12月期もコラボ商品継続し、客数伸長。
【DX】
  店員向けの表示板をガストなど約1700店に設置。回転率向上や従業員負担の軽減に寄与。テーブル決済は決済方法多様化、他社ポイント付与にも対応へ。

発行日: 2024年06月14日
【独自増額】
  店舗純増約30(前期純減80)。柱のガストは人気アニメとのコラボ販促効き、出足の客数想定超。注文点数増え客単価も上昇。食材高や人件費増こなす。のれん除却損減。前号比で営業益増額。税平常化。
【新中計】
  27年12月期売上4600億円、営業益320億円目標。既存店売上の年平均3～4%成長が達成のカギ。5月に自社ポイント導入。クーポン配布で再来店促進。

発行日: 2024年03月16日
【回復軌道】
  店舗純増約30(前期純減80)。広告宣伝強化や営業時間延長で客足伸長、注文点数増え客単価も上昇。台湾など海外好調。食材費や人件費増を吸収。のれん除却損も減り営業増益。税平常化。10円配。
【成長目標】
  国内は25年から3年で300程度出店、手薄な駅前や繁華街で拡大。4月から正社員を対象に平均約6.2%の賃上げ実施。初任給も引き上げ、採用を強化。

発行日: 2023年12月15日
【回復基調】
  店舗純減約70（前期42減）。各業態とも販促強化で注文点数増。食材大量購買など原価抑制。税効果減。24年12月期は店舗数微増。高単価商品投入で既存店堅調。のれん除却損減り営業増益。増配も。
【看板】
  

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

2024-03-31


code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3197,2024-12-31,"""予""",2024-08-14,395000,24000,21000,13000,57.1,17.5,4
3197,2024-12-31,"""予""",2024-02-14,375000,15000,12000,7500,33.0,10.0,4


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

すかいらーく HD(3197)の通期決算(評価日：2024年12月19日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2024年12月期(予)""",2024-08-14,395000,24000,21000,13000,57.1,17.5
"""2023年12月期""",2024-02-14,354831,11688,8691,4781,21.0,7.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3197,2024-09-30,"""四""",2024-11-13,103249,7305,6617,4162,18.3,7.1,3
3197,2024-06-30,"""四""",2024-08-14,95856,5883,5252,2873,12.6,6.1,2
3197,2024-03-31,"""四""",2024-05-15,95580,6100,5449,3421,15.0,6.4,1
3197,2023-12-31,"""四""",2024-02-14,91263,1737,715,248,1.1,1.9,4
3197,2023-09-30,"""四""",2023-11-10,93779,7098,6418,3984,17.5,7.6,3
3197,2023-06-30,"""四""",2023-08-10,85221,3233,2601,1210,5.3,3.8,2
3197,2023-03-31,"""四""",2023-05-15,84568,-380,-1043,-661,-2.9,-0.4,1
3197,2022-12-31,"""四""",2023-02-14,81835,-1395,-2021,-1755,-7.7,-1.7,4


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

すかいらーく HD(3197)の2024年12月期第3四半期決算進捗率(評価日：2024-12-19)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

すかいらーく HD(3197)の2023年12月期第3四半期決算進捗率(評価日：2023-12-19)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3197,2023-12-31,"""四""",2024-02-14,91263,1737,715,248,1.1,1.9,4
3197,2023-12-31,"""本""",2024-02-14,354831,11688,8691,4781,21.0,7.0,4
3197,2024-12-31,"""予""",2024-02-14,375000,15000,12000,7500,33.0,10.0,4
3197,2024-03-31,"""四""",2024-05-15,95580,6100,5449,3421,15.0,6.4,1
3197,2024-12-31,"""予""",2024-08-14,395000,24000,21000,13000,57.1,17.5,4
3197,2024-06-30,"""四""",2024-08-14,95856,5883,5252,2873,12.6,6.1,2
3197,2024-09-30,"""四""",2024-11-13,103249,7305,6617,4162,18.3,7.1,3


In [14]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [15]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)

code,date,unsold_margin,purchase_margin,margin_ratio
i64,date,i64,i64,f64
3197,2014-10-10,0,512900,inf
3197,2014-10-17,10000,324900,32.49
3197,2014-10-24,16800,516600,30.75
3197,2014-10-31,17500,443000,25.31
3197,2014-11-07,400,442600,1106.5
…,…,…,…,…
3197,2024-11-15,3149000,732800,0.23
3197,2024-11-22,4788300,693200,0.14
3197,2024-11-29,7724900,604400,0.08
